# Proyecto SQL

**Objetivo:** Generar una propuesta de valor para un nuevo producto en base a la base de datos sobre libros.

In [1]:
# importar librerías
import pandas as pd
from sqlalchemy import create_engine


db_config = {'user': 'practicum_student',         # nombre de usuario
             'pwd': 's65BlTKV3faNIGhmvJVzOqhs', # contraseña
             'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
             'port': 6432,              # puerto de conexión
             'db': 'data-analyst-final-project-db'}          # nombre de la base de datos

connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(db_config['user'],
                                                                     db_config['pwd'],
                                                                       db_config['host'],
                                                                       db_config['port'],
                                                                       db_config['db'])

engine = create_engine(connection_string, connect_args={'sslmode':'require'})

In [2]:
# Visualizar la tabla books
query = "SELECT * FROM books LIMIT 5;"

# Ejecutar la consulta y guardar el resultado en un DataFrame
books = pd.read_sql(query, con=engine)

# Mostrar los primeros 5 resultados
books.head()

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268


In [3]:
# Visualizar la tabla authors

query = "SELECT * FROM authors LIMIT 5;"

# Ejecutar la consulta y guardar el resultado en un DataFrame
authors = pd.read_sql(query, con=engine)

# Mostrar los primeros 5 resultados
authors.head()

,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd


In [4]:
# Visualizar la tabla publishers
query = "SELECT * FROM publishers LIMIT 5;"

# Ejecutar la consulta y guardar el resultado en un DataFrame
publishers = pd.read_sql(query, con=engine)

# Mostrar los primeros 5 resultados
publishers.head()

,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books
3,4,Ace Hardcover
4,5,Addison Wesley Publishing Company


In [5]:
# Visualizar la tabla ratings
query = "SELECT * FROM ratings LIMIT 5;"

# Ejecutar la consulta y guardar el resultado en un DataFrame
ratings = pd.read_sql(query, con=engine)

# Mostrar los primeros resultados
ratings.head()

,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2


In [6]:
# Visualizar la tabla reviews
query = "SELECT * FROM reviews LIMIT 5;"

# Ejecutar la consulta y guardar el resultado en un DataFrame
reviews = pd.read_sql(query, con=engine)

# Mostrar los primeros resultados
reviews.head()

,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...
3,4,3,johnsonamanda,Finally month interesting blue could nature cu...
4,5,3,scotttamara,Nation purpose heavy give wait song will. List...


**Número de libros publicados después del 1 de enero de 2000.**

In [7]:
query =  """
SELECT COUNT(DISTINCT title) 
FROM books 
WHERE publication_date > '2000-01-01';
"""

# Ejecutar la consulta y guardar el resultado en un DataFrame
num_books = pd.read_sql(query, con=engine)

# Mostrar los primeros resultados
num_books.head()

,count
0,818


El número de libros publicados después del primero de enero del 2000 son 818.

**Número de reseñas de usuarios y la calificación promedio para cada libro**

In [8]:
query = """
SELECT 
    ratings.book_id, 
    COUNT(reviews.review_id) AS num_reviews, 
    AVG(ratings.rating) AS avg_rating
FROM ratings
JOIN reviews ON ratings.book_id = reviews.book_id
GROUP BY ratings.book_id;
"""

df_result = pd.read_sql(query, engine)
df_result.head() 

,book_id,num_reviews,avg_rating
0,652,4,4.500000
1,273,4,4.500000
2,51,60,4.250000
3,951,4,4.000000
4,839,28,4.285714


El book_id 51, es el libro que más ha tenido review, y su calificación promedio de 60 usuarios es 4.25, lo que es positivo.

**Identificar la editorial que ha publicado el mayor número de libros con más de 50 páginas**

In [9]:
query = """
SELECT 
    publishers.publisher, 
    COUNT(books.book_id) AS num_books
FROM books
JOIN publishers ON books.publisher_id = publishers.publisher_id
WHERE books.num_pages > 50
GROUP BY publishers.publisher 
ORDER BY num_books DESC
LIMIT 1;
"""

df_result = pd.read_sql(query, engine)
df_result.head()

,publisher,num_books
0,Penguin Books,42


La editorial de Penguin Books es la editorial con mayor numero de libros con más de 50 páginas.

**Identificar al autor que tiene la más alta calificación promedio del libro: mirar solo los libros con al menos 50 calificaciones**

In [10]:
query = """
SELECT 
    authors.author, 
    AVG(ratings.rating) AS avg_rating
FROM authors
JOIN books ON authors.author_id = books.author_id
JOIN ratings ON books.book_id = ratings.book_id
WHERE books.book_id IN (
    SELECT book_id
    FROM ratings
    GROUP BY book_id
    HAVING COUNT(ratings.rating) >= 50
)
GROUP BY authors.author_id
ORDER BY avg_rating DESC
LIMIT 1;
"""

df_result = pd.read_sql(query, engine)
df_result.head()


,author,avg_rating
0,J.K. Rowling/Mary GrandPré,4.287097


J.K. Rowling/Mary GrandPré es la autora con la más alta calificación dada por los criticos literarios, como sugerencia, la empresa debe seguir promocionando sus libros y hacer campañas literarias con sus libros 

**Número promedio de reseñas de texto entre los usuarios que calificaron más de 50 libros**

In [11]:
query = """
SELECT 
    AVG(total_reviews) AS avg_review
FROM (
    SELECT 
        reviews.username, 
        COUNT(reviews.text) AS total_reviews
    FROM (
        SELECT ratings.username
        FROM ratings
        GROUP BY ratings.username
        HAVING COUNT(ratings.rating) >= 50
    ) AS filtered_users
    JOIN reviews ON filtered_users.username = reviews.username
    GROUP BY reviews.username
) AS user_reviews;
"""
df_result = pd.read_sql(query, engine)
df_result.head()

,avg_review
0,24.222222


El promedio de reseña de texto para un usuario que calificaron más de 50 libros es 24.22 reviews, esto quiere decir que hay un grupo considerable de criticos literarios que aman leer y compartir sus opiniones con el público. Este grupo es un gran nicho de mercado que se puede atender.